In [1]:
import docx2txt
import xlrd
import pandas as pd
import re 
import numpy as np
import os
from tqdm import tqdm
import sys
import copy
import requests, json


In [2]:
option_sheet_name = 'Sheet1'
option_sheet_index = 0

# option_sheet_name = 'Sheet2'
# option_sheet_index = 1


In [3]:
tmp_file_path = r"./tmp.txt"
seg_url = "http://101.6.15.212:8080/segmenter?text=%s"


In [4]:
excel = pd.read_excel(r"./data/decoration/装修手册_故障代码.xls",header=None,sheet_name=option_sheet_name)
excel

,0,1,2,3,4,5,6
0,故障代码 [989L00] 发动机控制器锁紧警告 1,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,报警级别,故障代码,故障,发动机控制器锁定警告 1,NaN,NaN,NaN
3,―,989L00,NaN,（机器监控器系统）,NaN,NaN,NaN
4,故障内容,· 检测到发动机控制器锁定（因素 1）。,NaN,NaN,NaN,NaN,NaN
5,控制器的反应,· 无特别反应。,NaN,NaN,NaN,NaN,NaN
6,NaN,· 如果故障原因消失，系统自身重设。,NaN,NaN,NaN,NaN,NaN
7,机器发生的故障,· 发动机不起动。,NaN,NaN,NaN,NaN,NaN
8,相关信息,· 如果在更换机器监控器后，显示此故障代码，用户密码必须换回更换之前的密码。,NaN,NaN,NaN,NaN,NaN
9,NaN,· 重新生成故障代码的方法: 将起动开关转到 ON 位置。,NaN,NaN,NaN,NaN,NaN


In [5]:
workbook = xlrd.open_workbook(r"./data/decoration/装修手册_故障代码.xls",formatting_info=True)

sheet = workbook.sheet_by_index(option_sheet_index)

In [6]:
synonymMap = {'报警级别':['报警级别'], '故障代码':['故障代码'], '故障':['故障'], '故障内容':['故障内容'],  \
              '控制器的反应':['控制器的反应'], '机器发生的故障':['机器发生的故障'],  \
             '相关信息':['相关信息'], '起因':['起因'], '诊断操作':['诊断操作'],'部件':['部件']}

name_to_en = {"故障代码":"faultCode", "故障名称":"faultName", "报警级别":"alertLevel","故障内容":"faultContent", 
                 "控制器的反应":"controllerReaction", "机器发生的故障":"machineFault", "相关信息":"relatedInfo", 
                 "起因":"cause", "诊断操作":"diagnosisAction", "标准值":"standardVal", "附图":"image", 
                  "检查系统":"monitorSystem", "故障":"faultName", "步骤数":"stepNum","部件":"part"}
name_to_en = {k: v.lower() for k, v in name_to_en.items()}
name_to_ch = {v: k for k, v in name_to_en.items()}
name_to_ch

{'faultcode': '故障代码',
 'faultname': '故障',
 'alertlevel': '报警级别',
 'faultcontent': '故障内容',
 'controllerreaction': '控制器的反应',
 'machinefault': '机器发生的故障',
 'relatedinfo': '相关信息',
 'cause': '起因',
 'diagnosisaction': '诊断操作',
 'standardval': '标准值',
 'image': '附图',
 'monitorsystem': '检查系统',
 'stepnum': '步骤数',
 'part': '部件'}

## 切分出每个故障

In [7]:
# 适用于一般的故障代码

code_line_num = []
for i in range(len(excel)):
#     print(i)
#     re.match(r'故障代码 \[\w*\].*',str(excel[i][0])) is not None:
    match_result = re.match(r'故障代码\s*\[([\w|@]*)\]\s*(.*)',str(excel.loc[i,0])) 
    if match_result is not None:
#         print(excel.loc[i,0])
#         print(match_result.group(1), match_result.group(2))
        code_line_num.append(i)
code_line_num.append(len(excel))
print(len(code_line_num))
code_line_num
        

128


[0,
 16,
 32,
 47,
 93,
 180,
 199,
 240,
 262,
 309,
 330,
 348,
 365,
 419,
 475,
 530,
 593,
 639,
 688,
 737,
 789,
 813,
 860,
 911,
 974,
 989,
 1007,
 1047,
 1100,
 1147,
 1202,
 1257,
 1312,
 1367,
 1422,
 1477,
 1494,
 1514,
 1553,
 1569,
 1615,
 1662,
 1700,
 1820,
 1836,
 1864,
 1919,
 1969,
 1988,
 2009,
 2061,
 2126,
 2162,
 2178,
 2290,
 2329,
 2367,
 2385,
 2399,
 2477,
 2607,
 2661,
 2749,
 2764,
 2784,
 2842,
 2909,
 3024,
 3131,
 3224,
 3300,
 3315,
 3331,
 3375,
 3394,
 3409,
 3431,
 3454,
 3482,
 3666,
 3716,
 3736,
 3806,
 3902,
 3970,
 4037,
 4107,
 4171,
 4240,
 4309,
 4372,
 4442,
 4511,
 4580,
 4649,
 4718,
 4735,
 4789,
 4833,
 4872,
 4913,
 5006,
 5104,
 5144,
 5204,
 5241,
 5281,
 5324,
 5365,
 5406,
 5446,
 5502,
 5556,
 5612,
 5666,
 5707,
 5747,
 5786,
 5826,
 5867,
 5908,
 5949,
 5990,
 6040,
 6090,
 6139,
 6189,
 6239]

In [8]:
len(code_line_num)

128

In [9]:
for line_num in code_line_num:
    print(excel.loc[line_num,0])

故障代码 [989L00]  发动机控制器锁紧警告 1
故障代码 [989M00]  发动机控制器锁紧警告 2
故障代码 [989N00]  发动机控制器锁紧警告 3
故障代码 [AA10NX] 空气滤清器堵塞
故障代码 [AB00KE] 充电电压低
故障代码 [B@BAZG] 机油压力低
故障代码 [B@BAZK] 机油油位低
故障代码 [B@BCNS] 发动机冷却液过热
故障代码 [B@BCZK] 散热器冷却液液位低
故障代码 [B@HANS] 液压油过热
故障代码 [CA111] ECM 临界内部故障
故障代码 [CA115] 发动机 Ne 和 Bkup 转速传感器故障
故障代码 [CA122] 增压压力传感器高压故障
故障代码 [CA123] 增压压力传感器低压故障
故障代码 [CA131] 油门传感器高压故障
故障代码 [CA132] 油门传感器低压故障
故障代码 [CA144] 冷却液温度传感器高压故障
故障代码 [CA145] 冷却液温度传感器低压故障
故障代码 [CA153] 增压温度传感器高压故障
故障代码 [CA154] 增压温度传感器低压故障
故障代码 [CA155] 增压温度高速下降
故障代码 [CA187] 传感器电源 2 电压低故障
故障代码 [CA221] 环境压力传感器高压故障
故障代码 [CA222] 环境压力传感器低压故障
故障代码 [CA227] 传感器电源 2 电压高故障
故障代码 [CA234] 发动机超速
故障代码 [CA238] Ne 转速传感器电源电压故障
故障代码 [CA271] IMV/PCV1 短路故障
故障代码 [CA272] IMV/PCV1 断路故障
故障代码 [CA322]  喷油器 #1（L#1）断路/短路故障
故障代码 [CA323]  喷油器 #5（L#5）断路/短路故障
故障代码 [CA324]  喷油器 #3（L#3）断路/短路故障
故障代码 [CA325]  喷油器 #6（L#6）断路/短路故障
故障代码 [CA331]  喷油器 #2（L#2）断路/短路故障
故障代码 [CA332]  喷油器 #4（L#4）断路/短路故障
故障代码 [CA342] 校准代码不兼容
故障代码 [CA351] 喷油器驱动电路故障
故障代码 [CA352] 传感器电源 1 电压低故障
故障代码 [CA386] 传

KeyError: 'the label [6239] is not in the [index]'

## 逐个读取出每个实体/属性的值

In [10]:
# 适用于一般的故障代码
fc_dicts = {}
for i_code in range(len(code_line_num)-1):
#     if i_code == 3:
#         break
    fc_dict = {}
    i_start = code_line_num[i_code]
    i_end = code_line_num[i_code+1]
#     print(str(excel.loc[i,0]))
    match_result = re.match(r'故障代码\s*\[([\w|@]*)\]\s*(.*)',str(excel.loc[i_start,0])) 
    fc_code = match_result.group(1)
    fc_content = match_result.group(2)
    print("i range in (%d,%d)"%(i_start, i_end))
#     print(excel.loc[i_start:i_end])

    for i in range(i_start, i_end):
        first_col_content = excel.loc[i,0]
        if first_col_content in synonymMap.keys():
#             print(first_col_content)
            if first_col_content == '报警级别':
                fc_dict[first_col_content] = excel.loc[i+1,0]
                for j in range(1,excel.shape[1]):
                    if excel.loc[i,j] == '故障代码':
#                         print(i+1)
#                         print(excel.loc[i+1,j])
                        fc_code_2 = re.match(r'\[?([\w|@]*)\]?',str(excel.loc[i+1,j])).group(1)
                        assert fc_code == fc_code_2, "%s %s"%(fc_code, fc_code_2)
                        fc_dict[excel.loc[i,j]] = fc_code
                    if excel.loc[i,j] == '故障':
                        if fc_content == excel.loc[i,j+1]:
                            fc_dict[excel.loc[i,j]] = fc_content
                        else:
                            print("WARNING：故障内容前后不一致：line_num=%d %s %s"%(i,fc_content, excel.loc[i,j+1]))
                            fc_dict[excel.loc[i,j]] = fc_content
                        req = requests.get(seg_url%fc_content)
                        fc_dict['部件'] = req.text.split()[0] 
                        
                        fc_dict['检查系统'] = excel.loc[i+1,j+1]
            if first_col_content == '故障内容':
#                 print(excel.shape)
#                 print(i_content)
#                 print(i)
#                 print(excel.loc[i_content,1])
                i_content = i 
                if excel.loc[i_content,1] != '.' and excel.loc[i_content,1] != '·' and excel.loc[i_content,1] == excel.loc[i_content,1]:
                    content = excel.loc[i,1]
                else:
                    content = excel.loc[i,2]
                fc_dict[first_col_content] = content
            if first_col_content == '控制器的反应':
                fc_dict[first_col_content] = [excel.loc[i,2]]
                for i_now in range(i+1,i+5):
                    if excel.loc[i_now,0] != '机器发生的故障':
                        if excel.loc[i_content,1] != '.' and excel.loc[i_content,1] != '·' and excel.loc[i_content,1] == excel.loc[i_content,1]:
                            content = excel.loc[i_now,1]
                        else:
                            content = excel.loc[i_now,2]
                        if content == content: # 检测是否为nan
                            fc_dict[first_col_content].append(content)
                    else:
                        break
            if first_col_content == '机器发生的故障':
                fc_dict[first_col_content] = [excel.loc[i,2]]
                for i_now in range(i+1,i+5):
                    if excel.loc[i_now+1,0] != '相关信息': # ‘相关信息’那几个字总是偏下一栏
                        if excel.loc[i_content,1] != '.' and excel.loc[i_content,1] != '·' and excel.loc[i_content,1] == excel.loc[i_content,1]:
                            content = excel.loc[i_now,1]
                        else:
                            content = excel.loc[i_now,2]
                        if content == content: # 检测是否为nan
                            fc_dict[first_col_content].append(content)
                    else:
                        break
            if first_col_content == '相关信息':
                
                for i_content_start in range(i,i-3,-1):
                    cell = sheet.cell(i_content_start,0)
                    fmt = workbook.xf_list[cell.xf_index]
                    file = open(tmp_file_path, mode='w')
                    fmt.dump(file)
                    file.close()
                    file = open(tmp_file_path,mode='r')
                    dump_dic = {}
                    for line in file.readlines():
                        match_result = re.match('(.*):\s(\d)', line)
                        if match_result is not None:
                            key = match_result.group(1).strip()
                            value = match_result.group(2)
                            dump_dic[key] = int(value)
                    file.close()
#                     print(dump_dic)
                    if dump_dic['bottom_line_style'] == 2:
                        break
                i_content_start = i_content_start + 1 # 往上多找了一行
                
                for i_content_end in range(i,i+5,1):
                    cell = sheet.cell(i_content_end,0)
                    fmt = workbook.xf_list[cell.xf_index]
                    file = open(tmp_file_path, mode='w')
                    fmt.dump(file)
                    file.close()
                    file = open(tmp_file_path,mode='r')
                    dump_dic = {}
                    for line in file.readlines():
                        match_result = re.match('(.*):\s(\d)', line)
                        if match_result is not None:
                            key = match_result.group(1).strip()
                            value = match_result.group(2)
                            dump_dic[key] = int(value)
                    file.close()
                    if dump_dic['bottom_line_style'] == 2:
                        break
#                 i_content_end += 1 # 
                
                fc_dict[first_col_content] = []
                for i_content in range(i_content_start,i_content_end+1): # +1是因为range函数会去掉后边界
                    if excel.loc[i_content,1] != '.' and excel.loc[i_content,1] != '·' and excel.loc[i_content,1] == excel.loc[i_content,1]:
                        content = excel.loc[i_content,1]
                    else:
                        content = excel.loc[i_content,2]
                    if content == content: # 检测是否为nan
                        fc_dict[first_col_content].append(content)
                        
            if first_col_content == '起因':
                split_line_num = []
                for i_content_end in range(i,i_end,1):
                    cell = sheet.cell(i_content_end,0)
                    fmt = workbook.xf_list[cell.xf_index]
                    file = open(tmp_file_path, mode='w')
                    fmt.dump(file)
                    file.close()
                    file = open(tmp_file_path,mode='r')
                    dump_dic = {}
                    for line in file.readlines():
                        match_result = re.match('(.*):\s(\d)', line)
                        if match_result is not None:
                            key = match_result.group(1).strip()
                            value = match_result.group(2)
                            dump_dic[key] = int(value)
                    file.close()
                    if dump_dic['bottom_line_style'] == 2:
                        split_line_num.append(i_content_end + 1)
#                 split_line_num.append(i_content_end)
#                 print(split_line_num)
                fc_dict[first_col_content] = []
                fc_dict['诊断操作'] = []
                fc_dict['标准值'] = []
                fc_dict['步骤数'] = []
                for i_split in range(len(split_line_num)-1):
                    key = ""
                    for i_content in range(split_line_num[i_split], split_line_num[i_split+1]):
                        if excel.loc[i_content,1] != '.' and excel.loc[i_content,1] != '·' and excel.loc[i_content,1] == excel.loc[i_content,1]:
                            key = excel.loc[i_content,1]
                            break
#                     value = ""
                    diagnosisActionList = []
                    standardValList = []
                    stepNumList = []
                    for i_content in range(split_line_num[i_split], split_line_num[i_split+1]):
                        value_count_in_this_row = 0
                        for j in range(2,excel.shape[1]):
                            value_now = excel.loc[i_content,j]
                            if value_now != '.' and value_now != '·' and value_now == value_now:
#                                 value += str(value_now)
                                value_count_in_this_row += 1
                        if value_count_in_this_row == 1:
                            step = excel.loc[i_content,2]
                            stepNum = 0
                            match_result = re.match(r'(\d)\.(.*)', str(step))
                            if match_result is not None:
                                step = match_result.group(2).strip()
                                stepNum = match_result.group(1)
                            diagnosisActionList.append(step)
                            stepNumList.append(int(stepNum))
                        else:
                            standardVal_line = ""
                            for j in range(2,excel.shape[1]):
                                value_now = excel.loc[i_content,j]
                                if value_now != '.' and value_now != '·' and value_now == value_now:
                                    standardVal_line += value_now
                            standardValList.append(standardVal_line)
                            
                    fc_dict[first_col_content].append(key)
                    fc_dict['诊断操作'].append(diagnosisActionList)
                    fc_dict['标准值'].append(standardValList)
                    fc_dict['步骤数'].append(stepNumList)
                    
#     print(fc_dict)
    fc_dicts[fc_dict['故障代码']] = fc_dict
fc_dicts
    
    

i range in (0,16)
WARNING：故障内容前后不一致：line_num=2 发动机控制器锁紧警告 1 发动机控制器锁定警告 1
i range in (16,32)
WARNING：故障内容前后不一致：line_num=18 发动机控制器锁紧警告 2 发动机控制器锁定警告 2
i range in (32,47)
WARNING：故障内容前后不一致：line_num=34 发动机控制器锁紧警告 3 发动机控制器锁定警告 3
i range in (47,93)
i range in (93,180)
i range in (180,199)
i range in (199,240)
i range in (240,262)
i range in (262,309)
WARNING：故障内容前后不一致：line_num=264 散热器冷却液液位低 散热器冷却水水位低
i range in (309,330)
i range in (330,348)
WARNING：故障内容前后不一致：line_num=332 ECM 临界内部故障 发动机控制器内部故障
i range in (348,365)
i range in (365,419)
i range in (419,475)
i range in (475,530)
i range in (530,593)
i range in (593,639)
i range in (639,688)
i range in (688,737)
i range in (737,789)
i range in (789,813)
i range in (813,860)
i range in (860,911)
i range in (911,974)
i range in (974,989)
i range in (989,1007)
i range in (1007,1047)
i range in (1047,1100)
i range in (1100,1147)
i range in (1147,1202)
WARNING：故障内容前后不一致：line_num=1149 喷油器 #1（L#1）断路/短路故障 喷油器#1（L#1）断路/短路故障
i range in (1202,1257)
WARNING：

{'989L00': {'报警级别': '―',
  '故障代码': '989L00',
  '故障': '发动机控制器锁紧警告 1',
  '部件': '发动机',
  '检查系统': '（机器监控器系统）',
  '故障内容': '· 检测到发动机控制器锁定（因素 1）。',
  '控制器的反应': [nan, '· 如果故障原因消失，系统自身重设。'],
  '机器发生的故障': [nan,
   '· 如果在更换机器监控器后，显示此故障代码，用户密码必须换回更换之前的密码。',
   '· 重新生成故障代码的方法: 将起动开关转到 ON 位置。'],
  '相关信息': ['· 如果在更换机器监控器后，显示此故障代码，用户密码必须换回更换之前的密码。',
   '· 重新生成故障代码的方法: 将起动开关转到 ON 位置。'],
  '起因': ['机器监控器的更换'],
  '诊断操作': [['此故障可能是由更换机器监控器引起。']],
  '标准值': [[]],
  '步骤数': [[0]]},
 '989M00': {'报警级别': '―',
  '故障代码': '989M00',
  '故障': '发动机控制器锁紧警告 2',
  '部件': '发动机',
  '检查系统': '（机器监控器系统）',
  '故障内容': '· 检测到发动机控制器锁定（因素 2）。',
  '控制器的反应': [nan, '· 如果故障原因消失，系统自身重设。'],
  '机器发生的故障': [nan,
   '· 如果在更换机器监控器后，显示此故障代码，用户密码必须换回更换之前的密码。',
   '· 重新生成故障代码的方法: 将起动开关转到 ON 位置。'],
  '相关信息': ['· 如果在更换机器监控器后，显示此故障代码，用户密码必须换回更换之前的密码。',
   '· 重新生成故障代码的方法: 将起动开关转到 ON 位置。'],
  '起因': ['机器监控器的更换', '机器监控器有问题'],
  '诊断操作': [['此故障可能是由更换机器监控器引起。'],
   ['当没有更换机器监控器时，显示此故障代码，机器监控器可能有问题。（由于 是内部故障，故不能进行故障诊断。）']],
  '标准值': [[], []],
  '步骤数': [[0], [

In [47]:
img_excel_path = r"E:/天远/知识图谱/天远知识图谱文档/天远百度/故障代码图片/故障代码_故障名称表.xlsx"
img_description = pd.read_excel(img_excel_path, header=None)
img_description['relative_path'] = img_description[0] + '_' + img_description[1]
img_description['faultCode'] = img_description[0].astype(str).apply(lambda s:s[1:-1])

count = 0
for index in img_description.index:
    code = img_description.loc[index, 'faultCode']
#     print(code)
    if code in fc_dicts.keys():
        fc_dicts[code]['附图'] = img_description.loc[index, 'relative_path']
        print(code)
        count += 1
fc_dicts

AA10NX
AB00KE
B@BAZK
B@BCZK
CA122
CA123
CA131
CA144
CA145
CA153
CA154
CA187
CA221
CA222
CA238
CA271
CA272
CA322
CA323
CA324
CA325
CA331
CA332
CA352
CA428
CA429
CA435
CA441
CA451
CA452
CA689
CA731
CA778
CA2185
CA2186
CA2555
CA2556
D110KB
D19JKZ
D862KA
DA22KK
DA25KP
DA26KP
DA29KQ
DA2QKR
DAF8KB
DA2QKR
DGH2KB
DHPAMA
DHPBMA
DHS3MA
DHS4MA
DHS8MA
DHS9MA
DHSAMA
DHSBMA
DHSCMA
DHSDMA
DHSFMA
DHSGMA
DHSHMA
DHSJMA
DR21KX
DV20KB
DW43KA
DW43KB
DW45KA
DW45KB
DW91KA
DW91KB
DWA2KA
DWA2KB
DWK0KA
DWK0KB
DWK2KA
DWK2KB
DXA8KA
DXA8KB
DXA9KA
DXA9KB
DXE0KA
DXE0KB
DXE4KA
DXE4KB
DXE5KA
DXE5KB
DXE6KA
DXE6KB
DY20KA
DY20MA
DY2CKB
DY2DKB
DY2EKB
D8AQKR


{'989L00': {'报警级别': '―',
  '故障代码': '989L00',
  '故障': '发动机控制器锁紧警告 1',
  '部件': '发动机',
  '检查系统': '（机器监控器系统）',
  '故障内容': '· 检测到发动机控制器锁定（因素 1）。',
  '控制器的反应': [nan, '· 如果故障原因消失，系统自身重设。'],
  '机器发生的故障': [nan,
   '· 如果在更换机器监控器后，显示此故障代码，用户密码必须换回更换之前的密码。',
   '· 重新生成故障代码的方法: 将起动开关转到 ON 位置。'],
  '相关信息': ['· 如果在更换机器监控器后，显示此故障代码，用户密码必须换回更换之前的密码。',
   '· 重新生成故障代码的方法: 将起动开关转到 ON 位置。'],
  '起因': ['机器监控器的更换'],
  '诊断操作': [['此故障可能是由更换机器监控器引起。']],
  '标准值': [[]],
  '步骤数': [[0]]},
 '989M00': {'报警级别': '―',
  '故障代码': '989M00',
  '故障': '发动机控制器锁紧警告 2',
  '部件': '发动机',
  '检查系统': '（机器监控器系统）',
  '故障内容': '· 检测到发动机控制器锁定（因素 2）。',
  '控制器的反应': [nan, '· 如果故障原因消失，系统自身重设。'],
  '机器发生的故障': [nan,
   '· 如果在更换机器监控器后，显示此故障代码，用户密码必须换回更换之前的密码。',
   '· 重新生成故障代码的方法: 将起动开关转到 ON 位置。'],
  '相关信息': ['· 如果在更换机器监控器后，显示此故障代码，用户密码必须换回更换之前的密码。',
   '· 重新生成故障代码的方法: 将起动开关转到 ON 位置。'],
  '起因': ['机器监控器的更换', '机器监控器有问题'],
  '诊断操作': [['此故障可能是由更换机器监控器引起。'],
   ['当没有更换机器监控器时，显示此故障代码，机器监控器可能有问题。（由于 是内部故障，故不能进行故障诊断。）']],
  '标准值': [[], []],
  '步骤数': [[0], [

In [42]:
'[AA10NX]_空气滤清器堵塞' in img_description['relative_path'].astype(str).tolist()

True

In [50]:
imgs = os.listdir( r"E:/天远/知识图谱/天远知识图谱文档/天远百度/故障代码图片/故障代码图片")
imgs = [img[:-4] for img in imgs]
print("存在该图片文件，但在excel中为未给出")
for img in imgs:
    if img not in img_description['relative_path'].tolist():
        print(img)
print()
print("excel中描述了该文件，但在图片文件夹中未找到")
for img in img_description['relative_path'].astype(str).tolist():
    if img not in imgs:
        print(img)
# imgs

存在该图片文件，但在excel中为未给出
[CA132]_油门传感器低压故障

excel中描述了该文件，但在图片文件夹中未找到
[CA132] _油门传感器低压故障


## 将故障值从字典形式转化为关系表形式

In [26]:
merge = pd.DataFrame(columns=name_to_en.keys())
for fc_dict_oral in fc_dicts.values():
#     print(fc_dict)
    fc_dict = copy.deepcopy(fc_dict_oral)
    list_keys = []
    for key in fc_dict.keys():
#         print(type(fc_dict[key]))
        if isinstance(fc_dict[key], list):
            list_keys.append(key)
#     print(list_keys)
    for key in list_keys:
        fc_dict.pop(key, None)
    single_fc_df = pd.DataFrame(fc_dict, index=[0])
    
    fc_dict = copy.deepcopy(fc_dict_oral)
    # fc_dict_list = {}
    for key in list_keys:
        if key == '诊断操作' or key == '标准值' or key == '步骤数':
            continue
        fc_dict_list = {}
#         print(fc_dict)
        fc_dict_list['故障代码'] = [fc_dict['故障代码']] * len(fc_dict[key])
        fc_dict_list[key] = fc_dict[key]
#         print(fc_dict_list)
#         print(pd.DataFrame(fc_dict_list))
        tmp_df = pd.DataFrame(fc_dict_list)
    #     a = pd.concat((a,b))
        single_fc_df = pd.concat((single_fc_df,tmp_df),sort=True )
    
    fc_dict = copy.deepcopy(fc_dict_oral)
    cause_dict = {}
    cause_dict['起因'] = []
    cause_dict['诊断操作'] = []
    cause_dict['步骤数'] = []
#     print(fc_dict)
    for i in range(len(fc_dict['起因'])):
        cause = fc_dict['起因'][i]
        diagnosisAction = fc_dict['诊断操作'][i]
        stepNum = fc_dict['步骤数'][i]
#         print("debug:cause=",cause)
#         print("debug:diagnosisAction=",diagnosisAction)
        cause_dict['起因'] += [cause] * len(diagnosisAction)
        cause_dict['诊断操作'] += diagnosisAction
        cause_dict['步骤数'] += stepNum
#         cause_dict['起因'].append(cause)
#         cause_dict['诊断操作'].append(diagnosisAction)
#     print(cause_dict)
    if len(cause_dict) == 1:
        tmp_df = pd.DataFrame(cause_dict, index=[0])
    else:
        tmp_df = pd.DataFrame(cause_dict)
    
    single_fc_df = pd.concat((single_fc_df,tmp_df),sort=True )
    merge = pd.concat((merge,single_fc_df), sort=True)
#     print("*"*80)
merge

,报警级别,控制器的反应,故障,故障代码,故障内容,故障名称,机器发生的故障,标准值,检查系统,步骤数,相关信息,诊断操作,起因,部件,附图
0,―,NaN,发动机控制器锁紧警告 1,989L00,· 检测到发动机控制器锁定（因素 1）。,NaN,NaN,NaN,（机器监控器系统）,NaN,NaN,NaN,NaN,发动机,NaN
0,NaN,NaN,NaN,989L00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,· 如果故障原因消失，系统自身重设。,NaN,989L00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,989L00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,989L00,NaN,NaN,· 如果在更换机器监控器后，显示此故障代码，用户密码必须换回更换之前的密码。,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,989L00,NaN,NaN,· 重新生成故障代码的方法: 将起动开关转到 ON 位置。,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,989L00,NaN,NaN,NaN,NaN,NaN,NaN,· 如果在更换机器监控器后，显示此故障代码，用户密码必须换回更换之前的密码。,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,989L00,NaN,NaN,NaN,NaN,NaN,NaN,· 重新生成故障代码的方法: 将起动开关转到 ON 位置。,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,989L00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,机器监控器的更换,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,此故障可能是由更换机器监控器引起。,机器监控器的更换,NaN,NaN


In [27]:
merge = merge.rename(columns=name_to_en)
merge.to_csv("./data/decoration/manual_fc.csv",quoting=1,sep=';',index=False,encoding='utf-16')
merge

,alertlevel,controllerreaction,faultname,faultcode,faultcontent,faultname,machinefault,standardval,monitorsystem,stepnum,relatedinfo,diagnosisaction,cause,part,image
0,―,NaN,发动机控制器锁紧警告 1,989L00,· 检测到发动机控制器锁定（因素 1）。,NaN,NaN,NaN,（机器监控器系统）,NaN,NaN,NaN,NaN,发动机,NaN
0,NaN,NaN,NaN,989L00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,· 如果故障原因消失，系统自身重设。,NaN,989L00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,989L00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,989L00,NaN,NaN,· 如果在更换机器监控器后，显示此故障代码，用户密码必须换回更换之前的密码。,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,989L00,NaN,NaN,· 重新生成故障代码的方法: 将起动开关转到 ON 位置。,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,989L00,NaN,NaN,NaN,NaN,NaN,NaN,· 如果在更换机器监控器后，显示此故障代码，用户密码必须换回更换之前的密码。,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,989L00,NaN,NaN,NaN,NaN,NaN,NaN,· 重新生成故障代码的方法: 将起动开关转到 ON 位置。,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,989L00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,机器监控器的更换,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,此故障可能是由更换机器监控器引起。,机器监控器的更换,NaN,NaN


In [35]:
s = '蓄电池电压低故障'

In [28]:
seg_url = "http://101.6.15.212:8080/segmenter?text=%s"
sequence = '铲斗卸载 PPC 压力传感器异常'
# data = json.dumps({'text':'蓄电池电压低故障'}) 
r = requests.get(seg_url%sequence)
# r.text.split()[0]

In [29]:
r.text

'铲斗 卸载 PPC 压力 传感器 异常'

In [53]:
r.text

'蓄电池 电 压低 故障'

In [57]:
r.text.split()

['蓄电池', '电', '压低', '故障']

In [58]:
"-".join(r.text.split())

'蓄电池-电-压低-故障'